## Detectron2 Instance Segmentation Inference of AVA DataSet

In [ ]:
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators
from detectron2.utils.visualizer import ColorMode
import os 
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2
import json 
from pycocotools import mask
from itertools import groupby
import numpy as np
from pathlib import Path
from detectron2.data import DatasetCatalog, MetadataCatalog
from model import *

In [ ]:
DatasetCatalog.clear()
Data_Resister_training="../Data_NoTemporal_cleaned_orig/train_xworld";
from detectron2.data.datasets import register_coco_instances

register_coco_instances(Data_Resister_training,{}, '../Data_NoTemporal_cleaned_orig/train_xworld.json', Path("../Data_NoTemporal_cleaned_orig/train_xworld"))

metadata = MetadataCatalog.get(Data_Resister_training)
dataset_train = DatasetCatalog.get(Data_Resister_training)

In [ ]:
cfg = get_cfg()
cfg.MODEL.ROI_MASK_HEAD.NAME='MaskRCNNConvUpsampleHead_multi'
config_name = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"#"COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml" #"COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml" # 
cfg.merge_from_file(model_zoo.get_config_file(config_name))
cfg.MODEL.ROI_MASK_HEAD.NUM_CONV=4

cfg.MODEL.WEIGHTS = os.path.join("../detectron_results/segmentation_101_multihead/model_0009999.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE='cpu'
cfg.MODEL.MASK_ON = True

In [ ]:
'''Following function is from detectron2 official github page -

https://github.com/facebookresearch/detectron2/blob/main/detectron2/evaluation/coco_evaluation.py

'''

from detectron2.structures import BoxMode
import pycocotools.mask as mask_util

def instances_to_coco_json(instances, img_id):
    """
    Dump an "Instances" object to a COCO-format json that's used for evaluation.
    Args:
        instances (Instances):
        img_id (int): the image id
    Returns:
        list[dict]: list of json annotations in COCO format.
    """
    num_instance = len(instances)
    if num_instance == 0:
        return []

    boxes = instances.pred_boxes.tensor.numpy()
    boxes = BoxMode.convert(boxes, BoxMode.XYXY_ABS, BoxMode.XYWH_ABS)
    boxes = boxes.tolist()
    scores = instances.scores.tolist()
    classes = instances.pred_classes.tolist()
    classes = [c + 1 for c in classes]

    has_mask = instances.has("pred_masks")
    if has_mask:
      
        rles = [
            mask_util.encode(np.array(mask[:, :, None], order="F", dtype="uint8"))[0]
            for mask in instances.pred_masks
        ]
        for rle in rles:
            
            rle["counts"] = rle["counts"].decode("utf-8")

    results = []
    for k in range(num_instance):
        result = {
            "image_id": int(img_id),
            "category_id": classes[k],
            "bbox": [float(b) for b in boxes[k]],
            "score": scores[k],
        }
        if has_mask:
            result["segmentation"] = rles[k]
     
        results.append(result)
    return results

In [ ]:

ann = {}

# Create predictor
predictor = DefaultPredictor(cfg)
all_results=[]

for file in os.listdir('../Data_NoTemporal/test_xworld/'):
    im = cv2.imread('../Data_NoTemporal/test_xworld/'+file)
    
    # Make prediction
    outputs = predictor(im)
    
    if outputs['instances'].pred_masks.numel() == 0:
        continue 

    else:
        coco_results= instances_to_coco_json(outputs['instances'].to('cpu'), file.split('.')[0])
        all_results.extend(coco_results)

print(all_results)
# Save the encoded data to a JSON file
with open('segmentation.json', 'w') as f:
    json.dump(all_results, f)

## Single Image Visualization

In [ ]:

im = cv2.imread("../Data_NoTemporal/test_xworld/020000006000.jpg")
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

In [ ]:
# Create predictor
predictor = DefaultPredictor(cfg)

# Make prediction
outputs = predictor(im)

In [ ]:
print(outputs['instances'])

In [ ]:
outputs['instances'].pred_masks.shape

In [ ]:
v = Visualizer(im[:, :, ::-1], metadata, scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))

In [ ]:
'''plot loss functions'''

In [ ]:
# import matplotlib.pyplot as plt
# import json

# json_files = ["segmentation_res_50_multihead/metrics.json", "segmentation_101_multihead/metrics.json", "renet-50-bam-residual-focaltaversky/metrics.json"]
# colors = ['c', 'm', 'y']
# labels = ['MHA-MaskRCNN_ResNet50_F.Tversky', 'MHA-MaskRCNN_ResNet101_F.Tversky', 'BAMS-MaskRCNN_ResNet50_F.Tversky']

# metrics = ['loss_mask', 'total_loss',  'loss_box_reg', 'fast_rcnn/cls_accuracy','mask_rcnn/false_negative', 'mask_rcnn/accuracy']

# for metric in metrics:
#     plt.figure(figsize=(10, 6))
#     for i, file_name in enumerate(json_files):
#         metric_values = []
#         with open('../detectron_results/'+file_name, 'r') as file:
#             for line in file:
#                 data = json.loads(line)
#                 if metric in data:
#                     metric_values.append(data[metric])
#                 else:
#                     metric_values.append(np.nan)  # Append NaN if key does not exist
        
#         plt.plot(metric_values, label=labels[i], color=colors[i])
#     plt.title(metric)
#     plt.legend()
#     plt.show()

